In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackathon-online-nlu-intent-classification/intent_sample_submission.csv
/kaggle/input/hackathon-online-nlu-intent-classification/train.json
/kaggle/input/hackathon-online-nlu-intent-classification/small_test.csv
/kaggle/input/hackathon-online-nlu-intent-classification/test.csv
/kaggle/input/hackathon-online-nlu-intent-classification/valid.json


In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 77.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 57.6 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=33d4476a78c7b36823a3de91acd1a5e4b3295543b24383a346359854490c6630
  Stored in directory: /root/.cache/pip/wheels/b2/a1/b7/0d3b008d0c77cd57332d724b92cf7650b4185b493dc785f00a
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=665515f8740a45afbfe5eda3c9c17415e7973b0d8d3a74c550670c38ac3abd69
  Stored in directory: /root/.cache/pip/wheels/3d/7c/f1/f87f51d07a2513aa0fb128d

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import simpletransformers

In [ ]:
sample=pd.read_csv("/kaggle/input/hackathon-online-nlu-intent-classification/intent_sample_submission.csv")
sample

,id,intent
0,1,AddToPlaylist
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN
...,...,...
295,296,NaN
296,297,NaN
297,298,NaN
298,299,NaN


In [ ]:
test=pd.read_csv("/kaggle/input/hackathon-online-nlu-intent-classification/test.csv")
test

,id,utterance
0,1,add this tune to my playlist guest list mashable
1,2,i d like to listen to the song the natural farmer
2,3,i need a table for 8 people at a restaurant in...
3,4,open the canciones del recuerdo playlist and p...
4,5,i want to hear something from the top-fifty by...
...,...,...
295,296,show weather forecast in west wildwood south c...
296,297,add this track to the playlist hits of the 70s
297,298,i want to eat at a food truck this week with m...
298,299,rate company for gertrude 4 of 6


In [ ]:
train=pd.read_json("/kaggle/input/hackathon-online-nlu-intent-classification/train.json")

In [ ]:
train

,utterance,slots,intent
0,will it be rainy in tenino,O O O B-condition_description O B-city,GetWeather
1,add steve albini album to my psychedelic rock ...,O B-artist I-artist B-music_item O B-playlist_...,AddToPlaylist
2,book a table for 3 at one of the restaurants i...,O O O O B-party_size_number O O O O B-restaura...,BookRestaurant
3,what s the weather forecast for belknap,O O O O O O B-city,GetWeather
4,play fereydoun farrokhzad best track,O B-artist I-artist B-sort B-music_item,PlayMusic
...,...,...,...
13079,show me the lights out saga,O O O B-object_name I-object_name B-object_type,SearchCreativeWork
13080,what is a weather of martinique a month from now,O O O O O B-country B-timeRange I-timeRange I-...,GetWeather
13081,i d like to reserve a spot at a bakery in guya...,O O O O O O O O O B-restaurant_type O B-countr...,BookRestaurant
13082,please find me the work serious awesomeness,O O O O O B-object_name I-object_name,SearchCreativeWork


In [ ]:
train.iloc[4]

utterance       play fereydoun farrokhzad best track
slots        O B-artist I-artist B-sort B-music_item
intent                                     PlayMusic
Name: 4, dtype: object

In [ ]:
train["intent"].value_counts()

GetWeather              1903
PlayMusic               1901
RateBook                1869
BookRestaurant          1862
SearchCreativeWork      1857
SearchScreeningEvent    1855
AddToPlaylist           1837
Name: intent, dtype: int64

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
train_t=train[["utterance","intent"]]
train_t

,utterance,intent
0,will it be rainy in tenino,GetWeather
1,add steve albini album to my psychedelic rock ...,AddToPlaylist
2,book a table for 3 at one of the restaurants i...,BookRestaurant
3,what s the weather forecast for belknap,GetWeather
4,play fereydoun farrokhzad best track,PlayMusic
...,...,...
13079,show me the lights out saga,SearchCreativeWork
13080,what is a weather of martinique a month from now,GetWeather
13081,i d like to reserve a spot at a bakery in guya...,BookRestaurant
13082,please find me the work serious awesomeness,SearchCreativeWork


In [ ]:
train_t["intent"]=train_t["intent"].replace("GetWeather",0)
train_t["intent"]=train_t["intent"].replace("PlayMusic",1)
train_t["intent"]=train_t["intent"].replace("RateBook",2)
train_t["intent"]=train_t["intent"].replace("BookRestaurant",3)
train_t["intent"]=train_t["intent"].replace("SearchCreativeWork",4)
train_t["intent"]=train_t["intent"].replace("SearchScreeningEvent",5)
train_t["intent"]=train_t["intent"].replace("AddToPlaylist",6)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
train_t

,utterance,intent
0,will it be rainy in tenino,0
1,add steve albini album to my psychedelic rock ...,6
2,book a table for 3 at one of the restaurants i...,3
3,what s the weather forecast for belknap,0
4,play fereydoun farrokhzad best track,1
...,...,...
13079,show me the lights out saga,4
13080,what is a weather of martinique a month from now,0
13081,i d like to reserve a spot at a bakery in guya...,3
13082,please find me the work serious awesomeness,4


In [ ]:
train_t.value_counts("intent")

intent
0    1903
1    1901
2    1869
3    1862
4    1857
5    1855
6    1837
dtype: int64

In [ ]:
from simpletransformers.classification import ClassificationModel
import sklearn
import torch
model_args = {
    "reprocess_input_data": True,
    'train_batch_size':5,
    "use_early_stopping": True,
    "early_stopping_delta": 0.01,
    "early_stopping_metric": "mcc",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 2,
    "evaluate_during_training_steps": 500,
    "fp16": False,
    "overwrite_output_dir":True,
    'use_cached_eval_features' : False, 
    'max_seq_length': 128,
    'no_cache': True,
    "warmup_ratio":0.06,
    "num_train_epochs":1,
    "adam_epsilon":1e-08,

}
model = ClassificationModel(
     "bert",
     "bert-base-uncased",
     num_labels=2,args=model_args,use_cuda=torch.cuda.is_available(),ignore_mismatched_sizes=True
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model.train_model(train_t, show_running_loss=True, acc=sklearn.metrics.accuracy_score)

/opt/conda/lib/python3.7/site-packages/simpletransformers/classification/classification_model.py:613: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/13084 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2617 [00:00<?, ?it/s]

(2617, 0.1526119145549944)

In [ ]:
test

,id,intent
0,1,AddToPlaylist
1,2,PlayMusic
2,3,BookRestaurant
3,4,AddToPlaylist
4,5,PlayMusic
...,...,...
295,296,GetWeather
296,297,AddToPlaylist
297,298,BookRestaurant
298,299,RateBook


In [ ]:
test

,id,utterance
0,1,add this tune to my playlist guest list mashable
1,2,i d like to listen to the song the natural farmer
2,3,i need a table for 8 people at a restaurant in...
3,4,open the canciones del recuerdo playlist and p...
4,5,i want to hear something from the top-fifty by...
...,...,...
295,296,show weather forecast in west wildwood south c...
296,297,add this track to the playlist hits of the 70s
297,298,i want to eat at a food truck this week with m...
298,299,rate company for gertrude 4 of 6


In [ ]:
test["utterance"].head(20)

0      add this tune to my playlist guest list mashable
1     i d like to listen to the song the natural farmer
2     i need a table for 8 people at a restaurant in...
3     open the canciones del recuerdo playlist and p...
4     i want to hear something from the top-fifty by...
5     i rate the mathematical magpie chronicle a 0 of 6
6     show the movie schedule of animated movies clo...
7     what time is careful  he might hear you playin...
8      i m looking to get a seat at a brasserie in togo
9         i need a table for 8 during midday in montana
10              play rain by subliminal on groove shark
11    what animated movies are playing close by at f...
12                      add album to bajo las estrellas
13                       i am giving this novel 3 stars
14    i need to episodes for the tv series  fast foo...
15    book a reservation for seven people at a steak...
16    reserve a spotat the food truck twelve weeks f...
17    where is the nearest movie theatre that is

In [ ]:
predictions, raw_outputs = model.predict(list(test["utterance"]))

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
predictions

array([6, 4, 3, 1, 1, 2, 5, 5, 3, 3, 1, 5, 6, 2, 4, 3, 3, 5, 6, 5, 1, 4,
       6, 2, 2, 2, 3, 0, 0, 4, 3, 5, 2, 3, 6, 4, 2, 6, 4, 5, 2, 3, 5, 3,
       3, 4, 0, 5, 5, 4, 3, 5, 6, 4, 3, 2, 4, 6, 5, 0, 0, 0, 2, 5, 5, 5,
       5, 0, 4, 6, 6, 1, 5, 0, 6, 3, 4, 2, 0, 6, 2, 0, 1, 3, 4, 5, 6, 2,
       5, 3, 1, 1, 2, 0, 1, 6, 2, 6, 3, 1, 2, 6, 0, 5, 1, 0, 5, 2, 6, 1,
       0, 6, 2, 2, 1, 1, 2, 1, 2, 0, 6, 3, 1, 3, 1, 1, 1, 6, 2, 6, 2, 6,
       2, 6, 3, 0, 4, 0, 6, 4, 2, 4, 6, 1, 6, 3, 4, 2, 1, 2, 0, 4, 6, 3,
       2, 3, 3, 4, 0, 1, 4, 4, 1, 0, 5, 6, 0, 0, 3, 6, 1, 6, 1, 3, 6, 3,
       6, 5, 1, 4, 1, 2, 6, 4, 4, 0, 6, 0, 6, 1, 2, 5, 5, 1, 1, 2, 6, 3,
       3, 4, 1, 5, 5, 3, 6, 1, 4, 3, 0, 3, 1, 5, 4, 5, 0, 2, 5, 0, 0, 0,
       1, 0, 5, 3, 2, 2, 3, 5, 5, 5, 6, 3, 1, 6, 4, 6, 1, 1, 3, 3, 4, 0,
       3, 1, 2, 0, 3, 2, 2, 6, 6, 6, 6, 5, 0, 5, 1, 2, 0, 4, 2, 5, 1, 6,
       6, 1, 1, 5, 3, 4, 2, 0, 6, 6, 0, 0, 0, 5, 3, 4, 1, 1, 4, 3, 4, 1,
       3, 0, 5, 0, 5, 5, 6, 3, 5, 0, 6, 3, 2, 3])

In [ ]:
test["intent"]=predictions

In [ ]:
test["intent"]=test["intent"].replace(0,"GetWeather")
test["intent"]=test["intent"].replace(1,"PlayMusic")
test["intent"]=test["intent"].replace(2,"RateBook")
test["intent"]=test["intent"].replace(3,"BookRestaurant")
test["intent"]=test["intent"].replace(4,"SearchCreativeWork")
test["intent"]=test["intent"].replace(5,"SearchScreeningEvent")
test["intent"]=test["intent"].replace(6,"AddToPlaylist")


In [ ]:
test=test.drop("utterance",axis=1)

In [ ]:
test.to_csv("bert_base_case.csv",index=False)
test

,id,intent
0,1,AddToPlaylist
1,2,SearchCreativeWork
2,3,BookRestaurant
3,4,PlayMusic
4,5,PlayMusic
...,...,...
295,296,GetWeather
296,297,AddToPlaylist
297,298,BookRestaurant
298,299,RateBook


In [ ]:
sample

,id,intent
0,1,AddToPlaylist
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN
...,...,...
295,296,NaN
296,297,NaN
297,298,NaN
298,299,NaN


#transformers